step 1 — Import libraries

In [ ]:
import pandas as pd


step 2 — Load the dataset

In [ ]:
df = pd.read_csv('../spotify_dataset.csv')  
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20594 entries, 0 to 20593
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist            20594 non-null  object 
 1   Track             20594 non-null  object 
 2   Album             20594 non-null  object 
 3   Album_type        20594 non-null  object 
 4   Danceability      20594 non-null  float64
 5   Energy            20594 non-null  float64
 6   Loudness          20594 non-null  float64
 7   Speechiness       20594 non-null  float64
 8   Acousticness      20594 non-null  float64
 9   Instrumentalness  20594 non-null  float64
 10  Liveness          20594 non-null  float64
 11  Valence           20594 non-null  float64
 12  Tempo             20594 non-null  float64
 13  Duration_min      20594 non-null  float64
 14  Title             20594 non-null  object 
 15  Channel           20594 non-null  object 
 16  Views             20594 non-null  float6

step 3 — Transform column types

In [ ]:

# STRING COLUMNS

string_cols = ['Artist', 'Track', 'Album', 'Title', 'Channel']
for col in string_cols:
    df[col] = df[col].astype(str)


# CATEGORY COLUMNS

category_cols = ['Album_type', 'most_playedon']
for col in category_cols:
    df[col] = df[col].astype('category')


# bool COLUMNS

# Keep only True, everything else becomes False, then convert type to bool
df['official_video'] = df['official_video'].apply(lambda x: True if x is True else False).astype(bool)



# FLOAT COLUMNS

float_cols = [
    'Danceability', 'Energy', 'Loudness', 'Speechiness',
    'Acousticness', 'Instrumentalness', 'Liveness',
    'Valence', 'Tempo', 'Duration_min', 'EnergyLiveness'
]

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# INTEGER COLUMNS (COUNTS)

int_cols = ['Views', 'Likes', 'Comments', 'Stream']
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')


# FINAL CHECK

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20594 entries, 0 to 20593
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Artist            20594 non-null  object  
 1   Track             20594 non-null  object  
 2   Album             20594 non-null  object  
 3   Album_type        20594 non-null  category
 4   Danceability      20594 non-null  float64 
 5   Energy            20594 non-null  float64 
 6   Loudness          20594 non-null  float64 
 7   Speechiness       20594 non-null  float64 
 8   Acousticness      20594 non-null  float64 
 9   Instrumentalness  20594 non-null  float64 
 10  Liveness          20594 non-null  float64 
 11  Valence           20594 non-null  float64 
 12  Tempo             20594 non-null  float64 
 13  Duration_min      20594 non-null  float64 
 14  Title             20594 non-null  object  
 15  Channel           20594 non-null  object  
 16  Views             2059

step 4 — Handle missing values

In [20]:
# Drop rows missing critical columns
df = df.dropna(subset=['Danceability','Energy','Loudness','Speechiness','Stream','Views'])

# Fill remaining numeric missing values with mean
numeric_cols = ['Danceability','Energy','Loudness','Speechiness','Acousticness',
                'Instrumentalness','Liveness','Valence','Tempo','Duration_min','Stream','Views','EnergyLiveness']
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


step 5 — Clean text columns

In [ ]:
# df['Artist'] = df['Artist'].str.title()
df['Track'] = df['Track'].str.title().str.replace('@','a')
df['Album_type'] = df['Album_type'].str.lower()


step 6 — Remove duplicates

In [22]:
df = df.drop_duplicates(subset=['Artist','Track','Album'])


step 7 — Add categorical columns

In [23]:
df['Total_popularity'] = df['Stream'] + df['Views']

Step 8: Drop unnecessary columns

In [24]:
columns_to_drop = ['Title', 'Channel', 'EnergyLiveness', 'Liveness', 
                   'Acousticness', 'Instrumentalness']
df = df.drop(columns=columns_to_drop, errors='ignore')

Step 9: Handle nulls in official_video

In [25]:
df['official_video'] = df['official_video'].replace(0, False).replace(1, True)


Step 10: Make values legible (minutes)

In [26]:
df['Duration_min'] = df['Duration_min'].round(1)
df['Danceability'] = (df['Danceability'] * 100).astype('int64')
df['Energy'] = (df['Energy'] * 100).astype('int64')
df['Loudness'] = df['Loudness'].round(1)
df['Speechiness'] = (df['Speechiness'] * 100 ).astype('int64')
df['Valence'] = (df['Valence'] * 100).astype('int64')
df['Tempo'] = df['Tempo'].astype('int64')

Step 11: Remove rows with duration > 20 min

In [27]:
df = df[df['Duration_min'] <= 20]

step 12: delete rows that have 0 stream

In [28]:
df['Stream'] = pd.to_numeric(df['Stream'], errors='coerce')
df = df[df['Stream'] > 0]

In [29]:
# ------------------------------
# Energy → 5 categories
# ------------------------------
def energy_category_5(e):
    if e < 20: return 'Very Low'
    elif e < 40: return 'Low'
    elif e < 60: return 'Medium'
    elif e < 80: return 'High'
    else: return 'Very High'

# ------------------------------
# Loudness → 5 categories
# ------------------------------
def loudness_category_5(l):
    if l < -20: return 'Very Soft'
    elif l < -15: return 'Soft'
    elif l < -10: return 'Medium'
    elif l < -5: return 'Loud'
    else: return 'Very Loud'

# ------------------------------
# Tempo → 5 categories
# ------------------------------
def tempo_category_5(t):
    if t < 70: return 'Very Slow'
    elif t < 90: return 'Slow'
    elif t < 110: return 'Medium'
    elif t < 130: return 'Fast'
    else: return 'Very Fast'

# ------------------------------
# Speechiness → 5 categories
# ------------------------------
def speechiness_category_5(s):
    if s < 10: return 'Very Musical'
    elif s < 20: return 'Musical'
    elif s < 40: return 'Mixed'
    elif s < 60: return 'Speechy'
    else: return 'Very Speechy'

# ------------------------------
# Valence → 5 categories
# ------------------------------
def valence_category_5(v):
    if v < 20: return 'Very Sad'
    elif v < 40: return 'Sad'
    elif v < 60: return 'Neutral'
    elif v < 80: return 'Happy'
    else: return 'Very Happy'

# ------------------------------
# Popularity (Stream + Views) → 5 categories
# ------------------------------
def popularity_category_5(s, v):
    total = s + v
    if total < 1e5: return 'Very Low'
    elif total < 5e5: return 'Low'
    elif total < 1e6: return 'Medium'
    elif total < 5e6: return 'High'
    else: return 'Very High'

# ------------------------------
# Apply to dataframe
# ------------------------------
df['Energy_level']       = df['Energy'].apply(energy_category_5)
df['Loudness_level']     = df['Loudness'].apply(loudness_category_5)
df['Tempo_level']        = df['Tempo'].apply(tempo_category_5)
df['Speechiness_level']  = df['Speechiness'].apply(speechiness_category_5)
df['Valence_level']      = df['Valence'].apply(valence_category_5)
df['Popularity_level']   = df.apply(lambda row: popularity_category_5(row['Stream'], row['Views']), axis=1)


step 13:slove the bool problem

In [30]:
def clean_boolean(x):

    if x in [True, 1, 'True', 'TRUE', 'true']:
        return True
    else:
        return False


df['official_video_clean'] = df['official_video'].apply(clean_boolean)

df['official_video'] = df['official_video_clean']
df['official_video'] = df['official_video'].astype(bool)  # ensure type
df = df.drop(columns=['official_video_clean'])  # remove helper column



In [31]:
df

,Artist,Track,Album,Album_type,Danceability,Energy,Loudness,Speechiness,Valence,Tempo,...,official_video,Stream,most_playedon,Total_popularity,Energy_level,Loudness_level,Tempo_level,Speechiness_level,Valence_level,Popularity_level
0,Gorillaz,Feel Good Inc.,Demon Days,album,81,70,-6.7,17,77,138,...,False,1040234854,Spotify,1733790075,High,Loud,Very Fast,Musical,Happy,Very High
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,67,70,-5.8,3,85,92,...,False,310083733,Spotify,382095378,High,Loud,Medium,Very Musical,Very Happy,Very High
2,Gorillaz,New Gold (Feat. Tame Impala And Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,69,92,-3.9,5,55,108,...,False,63063467,Spotify,71498522,Very High,Very Loud,Medium,Very Musical,Neutral,Very High
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,68,73,-5.8,2,57,120,...,False,434663559,Spotify,646418511,High,Loud,Fast,Very Musical,Neutral,Very High
4,Gorillaz,Clint Eastwood,Gorillaz,album,66,69,-8.6,17,52,167,...,False,617259738,Youtube,1235740696,High,Loud,Very Fast,Musical,Neutral,Very High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20589,Sick Legend,Just Dance Hardstyle,JUST DANCE HARDSTYLE,single,58,92,-6.3,3,65,90,...,False,9227144,Spotify,9298822,Very High,Loud,Medium,Very Musical,Happy,Very High
20590,Sick Legend,Set Fire To The Rain Hardstyle,SET FIRE TO THE RAIN HARDSTYLE,single,53,93,-1.8,13,65,174,...,False,10898176,Spotify,11062917,Very High,Very Loud,Very Fast,Musical,Happy,Very High
20591,Sick Legend,Outside Hardstyle Sped Up,OUTSIDE HARDSTYLE SPED UP,single,44,83,-4.7,6,41,168,...,False,6226110,Spotify,6261756,Very High,Very Loud,Very Fast,Very Musical,Neutral,Very High
20592,Sick Legend,Only Girl Hardstyle,ONLY GIRL HARDSTYLE,single,41,76,-4.0,41,53,155,...,False,6873961,Spotify,6880494,High,Very Loud,Very Fast,Speechy,Neutral,Very High


step 14 — Save the cleaned CSV

In [32]:
df.to_csv('../cleaned/cleaned_spotify_full_v10.csv', index=False)
print("✅ Cleaning done. Saved as 'cleaned_spotify_full_v10.csv'")

✅ Cleaning done. Saved as 'cleaned_spotify_full_v10.csv'
